In [1]:
import os

list_file = sorted(list(os.walk('./file/2.item'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.item/{file}',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Json_Compatibility'] != '{}'].reset_index(drop=True)

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：416


['235461387733',
 '233621258535',
 '334922305483',
 '255586148611',
 '234277033709',
 '232315264904',
 '235123315262',
 '256372876426',
 '235443864456',
 '256209081025',
 '256291287123',
 '335372891937',
 '235544392154',
 '335057592559',
 '232435277242',
 '332198105466',
 '235055850780',
 '232315242152',
 '255531675896',
 '232628030166',
 '235003975198',
 '234219682190',
 '335058269621',
 '255218432305',
 '235506777460',
 '256264372483',
 '334437920709',
 '233591979628',
 '335270739966',
 '256421824484',
 '235647995819',
 '255018389434',
 '335372988098',
 '256236108396',
 '256558512167',
 '234095593488',
 '234440531333',
 '334956110193',
 '256496980460',
 '334211431931',
 '332361055949',
 '235648003016',
 '255628697516',
 '335333813239',
 '256564411823',
 '235263199469',
 '256202118103',
 '235680200465',
 '234608488320',
 '256511996154',
 '256160146443',
 '255528777251',
 '255669513604',
 '235544303884',
 '334956170740',
 '335215802937',
 '256244140588',
 '256469386720',
 '334041548817

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import json
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

df_temp, json_, page = pd.DataFrame(), json.loads(input_.loc[a, 'Json_Compatibility']), 1
while page <= json_['page']:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            resp = requests.post(f'''{json_['api']}?module_groups=PART_FINDER&referrer=VIEWITEM&offset={page*20-20}&module=COMPATIBILITY_TABLE''',
                                 data=json.dumps(json_['data']),
                                 headers=get_header(ua=False, **{'Content-Type': 'application/json; charset=utf-8'}),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))

            if resp.status_code == 200:
                break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_header = [cell['textSpans'][0]['text'].strip() for cell in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']]

    # = = = = = = = = = = = = = = =

    list_row = [[cell['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in cell else cell['textSpans'][0]['text'].strip() for cell in row['cells']] for row in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]

    # = = = = = = = = = = = = = = =

    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                 'Item Number': input_.loc[a, 'Item Number'],
                                 'Json_Compatibility': input_.loc[a, 'Json_Compatibility'],
                                 'Page': page,
                                 'Row': [i+1 for i in range(len(list_row))]})

    for i, row in zip(range(len(list_row)), list_row):
        for j, header in zip(range(len(list_header)), list_header):
            df_temp_temp.loc[i, header] = row[j]

    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    page += 1

resp

1 1
1 2
1 3
1 4
1 5


<Response [200]>

In [8]:
len(df_temp) == 0

False

In [9]:
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Json_Compatibility,Page,Row,Year,Make,Model,Trim,Engine,Notes
0,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2009,Chevrolet,Trailblazer,LT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
1,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,2,2009,GMC,Envoy,SLE Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
2,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,3,2009,GMC,Envoy,SLT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
3,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,4,2009,Saab,9-7x,4.2i Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
4,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,5,2008,Chevrolet,Trailblazer,Base Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
...,...,...,...,...,...,...,...,...,...,...,...
95,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,16,2002,GMC,Envoy,SLE Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
96,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,17,2002,GMC,Envoy,SLT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
97,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,18,2002,GMC,Envoy XL,SLE Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
98,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,19,2002,GMC,Envoy XL,SLT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Item Number,Json_Compatibility,Page,Row,Year,Make,Model,Trim,Engine,Notes
0,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2009,Chevrolet,Trailblazer,LT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
1,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,2,2009,GMC,Envoy,SLE Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
2,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,3,2009,GMC,Envoy,SLT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
3,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,4,2009,Saab,9-7x,4.2i Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
4,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,5,2008,Chevrolet,Trailblazer,Base Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
...,...,...,...,...,...,...,...,...,...,...,...
95,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,16,2002,GMC,Envoy,SLE Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
96,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,17,2002,GMC,Envoy,SLT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
97,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,18,2002,GMC,Envoy XL,SLE Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,
98,1,235461387733,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",5,19,2002,GMC,Envoy XL,SLT Sport Utility 4-Door,4.2L 256Cu. In. l6 GAS DOHC Naturally Aspirated,


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
